<a href="https://colab.research.google.com/github/its-rajesh/Pattern-Recognition/blob/main/PRA1Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Speech Activity Detection**

### Imports

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


### Reading files

In [13]:
path = '/drive/My Drive/Pattern Recognition/Assignment1/assign1_data/q1/'
files = os.listdir(path)
files

['Segment3_cafe_noise1.wav',
 'Segment3_VAD_GT.csv',
 'Segment2_STEnergy.csv',
 'Segment3_MelEnergy.csv',
 'Segment3_STEnergy.csv',
 'Segment2_MelEnergy.csv',
 'Segment2_cafe_noise2.wav',
 'Segment2_VAD_GT.csv']

In [16]:
train_ste = pd.read_csv(path+'Segment2_STEnergy.csv')
train_mel = pd.read_csv(path+'Segment2_MelEnergy.csv')
train_gt = pd.read_csv(path+'Segment2_VAD_GT.csv')

train_ste.shape, train_mel.shape, train_gt.shape

((255, 1), (255, 1), (255, 1))

In [17]:
test_ste = pd.read_csv(path+'Segment3_STEnergy.csv')
test_mel = pd.read_csv(path+'Segment3_MelEnergy.csv')
test_gt = pd.read_csv(path+'Segment3_VAD_GT.csv')

test_ste.shape, test_mel.shape, test_gt.shape

((274, 1), (274, 1), (274, 1))